In [0]:
%pip install -r requirements.txt # Original had -U
dbutils.library.restartPython()

In [0]:
import yaml
import json
import requests

from databricks import agents
from databricks.vector_search.client import VectorSearchClient

with open("names.yaml", "r") as file:
    names = yaml.safe_load(file)
with open("rag_chain_config.yaml", "r") as file:
    config = yaml.safe_load(file)

VECTOR_SEARCH_ENDPOINT_NAME = names.get("vector_search_endpoint_name")
MODEL_NAME = names.get("model_name")
EXAMPLE = config.get("input_example")

In [0]:
vsc = VectorSearchClient()
endpoint_status = vsc.get_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME)['endpoint_status']['state']

if endpoint_status == 'ONLINE':
  print(f"Stopping endpoint {VECTOR_SEARCH_ENDPOINT_NAME}.")
  vsc.stop_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME)
else:
  print(f"Endpoint: {VECTOR_SEARCH_ENDPOINT_NAME} {endpoint_status}. Starting endpoint.")
  vsc.start_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME)

In [0]:
auth_token = 'IINSERT_TOKEN'
headers = {"Authorization": f"Bearer {auth_token}"}

model_endpoint = agents.get_deployments(MODEL_NAME)[-1].query_endpoint
model_query = requests.post(model_endpoint, json=EXAMPLE, headers=headers)
if model_query.ok:
  print('Sent request to model endpoint')
else:
  print(f'Response not received. Code {model_query.status_code}')